# File paths and general generators

In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import cv2
import os
import random
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
d1_base_path = "./COVIDvsARDS/main/"
d2_base_path = "./COVIDvsNORMAL/main/"
d3_base_path = "./COVIDvsVBPNEUMONIA/main/"

In [4]:
INIT_LR = 1e-3
EPOCHS = 25
BS = 8

In [19]:
def getImagesAndLabels(filepath, res):
    
    print("[INFO] loading images and labels...")
    imagePaths = list(paths.list_images(filepath))
    data = []
    labels = []

    # loop over the image paths
    for imagePath in imagePaths:
        # extract the class label from the filename
        label = imagePath.split(os.path.sep)[-2]

        # load the image, swap color channels, and resize it to be a fixed
        # 224x224 pixels while ignoring aspect ratio
        image = cv2.imread(imagePath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (res, res))

        # update the data and labels lists, respectively
        data.append(image)
        labels.append(label)

    # convert the data and labels to NumPy arrays while scaling the pixel
    # intensities to the range [0, 255]
    data = np.array(data) / 255.0
    labels = np.array(labels)

    # perform one-hot encoding on the labels
    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)
    labels = tf.keras.utils.to_categorical(labels)
    
    print("Data: ", end="")
    print(data.shape)
    print("Labels: ", end="")
    print(labels.shape)
    
    return data, labels

In [6]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,
                                                                rotation_range=20,
                                                                zoom_range=0.2,                 
                                                                horizontal_flip = True ,
                                                                fill_mode="nearest")

# VGG16 Model

In [21]:
vgg16_weightspath = "./weights/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"

vgg16baseModel = tf.keras.applications.VGG16(weights=None, include_top=False,input_shape=(256,256,3))

vgg16baseModel.load_weights(vgg16_weightspath)

x = vgg16baseModel.output
x = tf.keras.layers.Conv2D(16, (3,3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D(2,2)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256, activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(2, activation="softmax")(x)

for layer in vgg16baseModel.layers:
    layer.trainable = False

vgg16_model = tf.keras.Model(vgg16baseModel.input, x)
vgg16_model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

vgg16_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0   

In [29]:
# load the VGG16 network, ensuring the head FC layer sets are left
# off
baseModel = tf.keras.applications.VGG16(weights="imagenet", include_top=False,input_shape=(256,256,3))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = tf.keras.layers.AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = tf.keras.layers.Flatten(name="flatten")(headModel)
headModel = tf.keras.layers.Dense(64, activation="relu")(headModel)
headModel = tf.keras.layers.Dropout(0.5)(headModel)
headModel = tf.keras.layers.Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = tf.keras.Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

# compile our model
print("[INFO] compiling model...")
opt = tf.keras.optimizers.Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

print("DONE")

[INFO] compiling model...


# D2 normal

In [ ]:
# vgg16

In [20]:
d2_images, d2_labels = getImagesAndLabels(d2_base_path, 256)

[INFO] loading images and labels...
Data: (156, 256, 256, 3)
Labels: (156, 2)


In [22]:
(trainX, testX, trainY, testY) = train_test_split(d2_images, d2_labels,
	test_size=0.20, stratify=d2_labels, random_state=62)

In [14]:
trainX.shape

(124, 224, 224, 3)

In [15]:
testX.shape

(32, 224, 224, 3)

In [16]:
testY.shape

(32, 2)

In [30]:
# train the head of the network
print("[INFO] training head...")
H = model.fit_generator(
	train_datagen.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=10)

[INFO] training head...
  ...
    to  
  ['...']
Train for 15 steps, validate on 32 samples
Epoch 1/10
15/15 [==============================] - 56s 4s/step - loss: 0.8419 - accuracy: 0.4483 - val_loss: 0.6868 - val_accuracy: 0.5938
Epoch 2/10
15/15 [==============================] - 62s 4s/step - loss: 0.7584 - accuracy: 0.4828 - val_loss: 0.6852 - val_accuracy: 0.5000
Epoch 3/10
15/15 [==============================] - 60s 4s/step - loss: 0.7327 - accuracy: 0.4655 - val_loss: 0.6883 - val_accuracy: 0.5312
Epoch 4/10
15/15 [==============================] - 60s 4s/step - loss: 0.7112 - accuracy: 0.4741 - val_loss: 0.6675 - val_accuracy: 0.7812
Epoch 5/10
15/15 [==============================] - 60s 4s/step - loss: 0.7249 - accuracy: 0.5000 - val_loss: 0.6701 - val_accuracy: 0.6250
Epoch 6/10
15/15 [==============================] - 68s 5s/step - loss: 0.7031 - accuracy: 0.5603 - val_loss: 0.6795 - val_accuracy: 0.5000
Epoch 7/10
15/15 [==============================] - 65s 4s/step - lo